# Muptipage Panel | Hvplot dashboard

### Inputs
```
# Load dataset
preprocessed_df = pd.read_feather('../output/nb2_market_index_with_historical_and_features.feather')

# Load trained model
model = joblib.load('../output/xgb_model.pkl')

# Load fitted scaler
scaler = joblib.load('../output/scaler.pkl')
```

### Outputs
Multipage dashboard

### Notes / comments

---

In [ ]:
# import modules
import joblib
import numpy as np
import panel as pn
import pandas as pd
import hvplot.pandas
import warnings
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# set warnings
warnings.filterwarnings('ignore')

#### Prepare prediction

In [ ]:
# Load dataset
preprocessed_df = pd.read_feather('../output/nb2_market_index_with_historical_and_features.feather')

# Load trained model
model = joblib.load('../output/xgb_model.pkl')

# Load fitted scaler
scaler = joblib.load('../output/scaler.pkl')

# Prepare the DataFrame
model_df = preprocessed_df.dropna().copy().reset_index().rename(columns={"index": "Date"})
model_df["Date"] = pd.to_datetime(model_df["Date"])

# Define Features & Target
feature_cols = ["Has_Event", "SMA_4", "RSI_4", "MACD", "MACD_Signal", "BB_Width"]
X = model_df[feature_cols]
y = model_df["Sentiment_Label"]

# Spllit dataset (time-wise)

split_index = int(len(model_df) * 0.8)
train_df = model_df.iloc[:split_index]
test_df = model_df.iloc[split_index:]

X_train = train_df[feature_cols]
y_train = train_df["Sentiment_Label"]
X_test = test_df[feature_cols]
y_test = test_df["Sentiment_Label"]

# Scaling X_test
X_test_scaled = scaler.transform(X_test)

# Predict on Test Set Only
y_pred = model.predict(X_test_scaled)

# Add Predictions to model_df (only for test part)
model_df["Predicted_Label"] = np.nan  # initialize with NaNs
model_df.loc[split_index:, "Predicted_Label"] = y_pred

# Predict probabilities (confidence that label is bullish)
y_proba = model.predict_proba(X_test_scaled)[:, 1]  # Probability of class 1 (bullish)

# Store in the test portion of model_df
model_df.loc[split_index:, "Confidence_Score"] = y_proba

# Mark correctness
model_df["Correct"] = np.nan
model_df.loc[split_index:, "Correct"] = (
    model_df.loc[split_index:, "Predicted_Label"] == model_df.loc[split_index:, "Sentiment_Label"]
)

#### Create charts

In [ ]:
# Here go functions for creating charts, graphs and diagrams
# This is a placeholder

#### Widgets and Callbacks

In [ ]:
# Here go functions for creating widgets and corresponding callbacks
# Source of icons: https://tabler-icons.io/

In [ ]:
# Sidebar buttons
button_1 = pn.widgets.Button(name="Predicted Sentiment", button_type="primary", button_style="outline", icon="", styles={'width': '90%'})
button_2 = pn.widgets.Button(name="Info", button_type="primary", button_style="outline", icon="", styles={'width': '90%'})
button_3 = pn.widgets.Button(name="Graphs", button_type="primary", button_style="outline", icon="", styles={'width': '90%'})
button_4 = pn.widgets.Button(name="Model Performance", button_type="primary", button_style="outline", icon="", styles={'width': '90%'})

In [ ]:
# Select what to be shown on the graph, page_3
# multi_select = pn.widgets.MultiSelect(name="Ingredients", options=wine.feature_names, value=["alcohol", "malic_acid", "ash"])

In [ ]:
disclaimer = pn.pane.Markdown(
        f"""
	##### <u>Disclaimer</u>: The information and predictions presented in this project are for educational and informational purposes only. They do not constitute investment advice, financial guidance, or a recommendation to buy or sell any securities. While efforts have been made to ensure the accuracy of the data and models used, no guarantee is provided regarding the reliability, completeness, or future performance of any predictions. Always conduct your own research and consult with a licensed financial advisor before making any investment decisions. The creators of this project are not responsible for any financial losses incurred as a result of using or relying on the content presented.""",
	sizing_mode="stretch_width")

In [ ]:
# creating page content
def create_page_1():
    """
    shows current (live) lithium index sentiment
    """
    # scaler = StandardScaler()
    model = joblib.load('../output/xgb_model.pkl')
    scaler = joblib.load('../output/scaler.pkl')
    feature_cols = ["Has_Event", "SMA_4", "RSI_4", "MACD", "MACD_Signal", "BB_Width"]
    preprocessed_df = pd.read_feather('../output/nb2_market_index_with_historical_and_features.feather')
    X_next = preprocessed_df.dropna().copy().tail(1)[feature_cols]
    X_next_scaled = scaler.transform(X_next)
    next_label = model.predict(X_next_scaled)[0]
    next_proba = model.predict_proba(X_next_scaled)[0][1]
    label_str = "📈 Bullish" if next_label == 1 else "📉 Bearish"
    confidence_str = f"{next_proba:.2%}"
    forecast_card = pn.pane.Markdown(
    f"""
	### 🔮 **Next Week Forecast**
	- **Sentiment:** {label_str}
	- **Confidence (of being Bullish):** {confidence_str}
	""",
    sizing_mode="stretch_width")
    
    hr = pn.pane.Markdown("<hr>", sizing_mode="stretch_width")
        
    return pn.Column(
        pn.pane.Markdown("## 🔋Next week predicted Li sentiment"),
        forecast_card,
        hr,
        disclaimer,
        align="center")


def create_page_2():
    """
    shows information about the project
    """
    info_card = pn.pane.Markdown(
    f"""
	- The **App** predicts if the next week lithium index market sentiment is **bullish** or **bearish**
	- Prediction is made using trained XGBoost classifier
	- Training dataset combines:
		- Historical market data from [yfinance](https://pypi.org/project/yfinance/) (calculated lithium index)
		- Features engineered using technical analysis
		- Library of influential hystorical events (Li-battery-specific)
	""",
    sizing_mode="stretch_width")
            
    return pn.Column(
        pn.pane.Markdown("## About the project", sizing_mode="stretch_width"),
        pn.pane.Markdown("MVP - dashboard showing model applicability", sizing_mode="stretch_width"),
        pn.pane.Markdown("[Project repo](https://github.com/DrSYakovlev/lithium-market-prediction)"),
        info_card,
        disclaimer,
        align="center")


def create_page_3():
    """
    shows time series graph
    """
    preprocessed_df = pd.read_feather('../output/nb2_market_index_with_historical_and_features.feather')
    date_slider = pn.widgets.DateRangeSlider(
        name='Date Range',
        start=preprocessed_df.index.min(),
        end=preprocessed_df.index.max(),
        value=(preprocessed_df.index.min(),
               preprocessed_df.index.max()))
    
    # Interactive plot
    @pn.depends(date_slider.param.value)
    def sentiment_plot(date_range):
        start = pd.to_datetime(date_range[0])
        end = pd.to_datetime(date_range[1])
        df = model_df[(model_df["Date"] >= start) & (model_df["Date"] <= end)]
        # Base line plot
        plot = df.hvplot.line(x="Date", y="Lithium_Market_Index", label="Lithium Index", line_width=2, color="dodgerblue")
        
        # Actual sentiment markers
        bull = df[df["Sentiment_Label"] == 1].hvplot.scatter(x="Date", y="Lithium_Market_Index", color="green", size=40, marker="^", label="Bullish")
        bear = df[df["Sentiment_Label"] == 0].hvplot.scatter(x="Date", y="Lithium_Market_Index", color="red", size=40, marker="v", label="Bearish")
        
        # Predicted bullish markers (only in test portion)
        pred_bull = df[df["Predicted_Label"] == 1].hvplot.scatter(x="Date", y="Lithium_Market_Index", color="blue", marker="*", size=100, label="Predicted Bullish")
        
        # Incorrect predictions (predicted bullish, actually bearish)
        incorrect = df[(df["Predicted_Label"] == 1) & (df["Correct"] == False)].hvplot.scatter(x="Date", y="Lithium_Market_Index", color="yellow", marker="x", size=100, label="Incorrect Prediction")
        return plot * bull * bear * pred_bull * incorrect
    
    
    return pn.Column(
        pn.pane.Markdown("## Explore sentiment predictions by the model"),
        date_slider,
        sentiment_plot,
        pn.pane.Markdown('<hr>', sizing_mode="stretch_width"),
        disclaimer,
        align="center")


def create_page_4():
    """
    shows model performance
    """
    # Calculate confusion matrix on test set
    conf_mat = confusion_matrix(y_test, y_pred, labels=[0, 1])
    
    # Convert to long format
    cm_long = pd.DataFrame(conf_mat, index=["Actual Bearish", "Actual Bullish"], columns=["Predicted Bearish", "Predicted Bullish"])
    cm_long = cm_long.reset_index().melt(id_vars="index", var_name="Predicted", value_name="Count")
    cm_long = cm_long.rename(columns={"index": "Actual"})
    
    cm_plot = cm_long.hvplot.heatmap(
        x='Predicted',
        y='Actual',
        C='Count',
        cmap='Blues',
        line_color='white',
        # title='Confusion Matrix (Test Set)',
        colorbar=True,
        height=300, width=450)
    
    # Precision / Recall / f1
    report_dict = classification_report(y_test, y_pred, target_names=["Bear", "Bull"], output_dict=True)
    # Convert it to a DataFrame and melt it into long form
    report_df = pd.DataFrame(report_dict).T.iloc[:2]  # Only Bearish & Bullish
    report_df['Class'] = report_df.index
    report_long = report_df.melt(
        id_vars="Class",
        value_vars=["precision", "recall", "f1-score"],
        var_name="Metric",
        value_name="Score")
    
    report_bar = report_long.hvplot.bar(
        x="Metric",
        y="Score",
        by="Class",
        rot=0,
        ylabel="Score",
        xlabel="Metric",
        # title="Model Performance Metrics (Test Set)",
        height=350,
        width=600,
        legend='top_right',
        color=['#1f77b4', '#ff7f0e'])    
        
    return pn.Column(
        pn.pane.Markdown("## Model performance"),
        pn.pane.Markdown("### 🧠 Confusion matrix (test set)"),
        cm_plot,
        pn.pane.Markdown('<hr>', sizing_mode="stretch_width"),
        pn.pane.Markdown("### 📊 Precision / Recall / F1-Score"),
        report_bar,
        pn.pane.Markdown('<hr>', sizing_mode="stretch_width"),
        disclaimer,
        align="center")


#### Buttons - calling (switching between) pages

In [ ]:
# Page mapping to functions
mapping = {
    "Page1": create_page_1(),
    "Page2": create_page_2(),
    "Page3": create_page_3(),
    "Page4": create_page_4()
}


In [ ]:
# main area
main_area = pn.Column(mapping["Page1"], styles={"width":"100%"})

In [ ]:
def show_page(page_key):
    """
    callback function for all buttons
    """
    main_area.clear()  # It will clear the main area each time function is called
    main_area.append(mapping[page_key])  # and the new page will be appended


In [ ]:
# calling actions when buttons is clicked
button_1.on_click(lambda event: show_page("Page1"))
button_2.on_click(lambda event: show_page("Page2"))
button_3.on_click(lambda event: show_page("Page3"))
button_4.on_click(lambda event: show_page("Page4"))


#### Layout

In [ ]:
# sidebar
sidebar = pn.Column(pn.pane.Markdown("## Pages"), button_1, button_2, button_3, button_4,
					styles={"width": "100%", "padding": "15px"})

#### App layout

In [ ]:
# App layout
template = pn.template.BootstrapTemplate(
    title="Lithium Market Sentiment Dashboard",
    sidebar=[sidebar],
    main=[main_area],
    header_background="black", 
    # site="CoderzColumn", logo="cc.png",
    theme=pn.template.DarkTheme,
    sidebar_width=250, ## Default is 330
    busy_indicator=None,
)

#### Run app locally

In [ ]:
template.show()